### NCE loss with fixing negative data

In [1]:
import numpy as np
import tensorflow as tf
from util import print_tf_tensor,gpu_sess
np.set_printoptions(precision=3)
print ("Done.")

Done.


### Data

In [7]:
n_batch,dim,n_neg = 3,2,5
bias = np.random.randn(n_batch,dim)
eps = 0.1
ha = tf.cast(tf.Variable(bias+eps*np.random.randn(n_batch,dim)),tf.float32,name='ha')
hb = tf.cast(tf.Variable(bias+eps*np.random.randn(n_batch,dim)),tf.float32,name='hb')
neg = tf.cast(tf.Variable(np.random.randn(n_neg,dim)),tf.float32,name='neg')

In [8]:
sess = gpu_sess()
sess.run(tf.global_variables_initializer())
print_tf_tensor(sess,ha)
print_tf_tensor(sess,hb)
print_tf_tensor(sess,neg)

name:[ha:0] shape:(3, 2)
[[-9.833e-01  8.341e-01]
 [ 8.658e-01 -4.415e-05]
 [-2.182e+00 -6.825e-01]]
name:[hb:0] shape:(3, 2)
[[-0.68   0.688]
 [ 0.79  -0.135]
 [-2.138 -0.649]]
name:[neg_1:0] shape:(5, 2)
[[-0.354  0.031]
 [ 0.304 -0.457]
 [-0.677  1.099]
 [ 1.348 -1.866]
 [ 0.689  0.571]]


### Loss

In [9]:
ha_nzd = tf.math.l2_normalize(ha,axis=-1)
hb_nzd = tf.math.l2_normalize(hb,axis=-1)
neg_nzd = tf.math.l2_normalize(neg,axis=-1)
print_tf_tensor(sess,ha_nzd)
print_tf_tensor(sess,hb_nzd)
print_tf_tensor(sess,neg_nzd)

name:[l2_normalize_3:0] shape:(3, 2)
[[-7.626e-01  6.469e-01]
 [ 1.000e+00 -5.099e-05]
 [-9.544e-01 -2.986e-01]]
name:[l2_normalize_4:0] shape:(3, 2)
[[-0.703  0.711]
 [ 0.986 -0.169]
 [-0.957 -0.291]]
name:[l2_normalize_5:0] shape:(5, 2)
[[-0.996  0.087]
 [ 0.553 -0.833]
 [-0.525  0.851]
 [ 0.586 -0.811]
 [ 0.77   0.638]]


In [10]:
temperature = 1.0
logits_ab = tf.matmul(ha_nzd/temperature,hb_nzd,transpose_b=True,name='logits_ab')
logits_ba = tf.matmul(hb_nzd/temperature,ha_nzd,transpose_b=True,name='logits_ba')
logits_an = tf.matmul(ha_nzd/temperature,neg_nzd,transpose_b=True,name='logits_an') 
logits_bn = tf.matmul(hb_nzd/temperature,neg_nzd,transpose_b=True,name='logits_bn') 
print_tf_tensor(sess,logits_ab)
print_tf_tensor(sess,logits_ba)
print_tf_tensor(sess,logits_an)
print_tf_tensor(sess,logits_bn)

name:[logits_ab_1:0] shape:(3, 3)
[[ 0.996 -0.861  0.542]
 [-0.703  0.986 -0.957]
 [ 0.458 -0.89   1.   ]]
name:[logits_ba_1:0] shape:(3, 3)
[[ 0.996 -0.703  0.458]
 [-0.861  0.986 -0.89 ]
 [ 0.542 -0.957  1.   ]]
name:[logits_an_1:0] shape:(3, 5)
[[ 0.816 -0.961  0.951 -0.971 -0.174]
 [-0.996  0.553 -0.525  0.586  0.77 ]
 [ 0.925 -0.279  0.247 -0.317 -0.925]]
name:[logits_bn_1:0] shape:(3, 5)
[[ 0.762 -0.981  0.974 -0.988 -0.087]
 [-0.997  0.686 -0.661  0.714  0.651]
 [ 0.928 -0.287  0.255 -0.325 -0.922]]


In [11]:
labels = tf.one_hot(tf.range(n_batch),n_batch+n_neg,name='labels')
print_tf_tensor(sess,labels)

name:[labels_1:0] shape:(3, 8)
[[1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0.]]


In [18]:
weights = 1.0
loss_a = tf.losses.softmax_cross_entropy(
  labels, tf.concat([logits_ab, logits_an], 1), weights=weights)
loss_b = tf.losses.softmax_cross_entropy(
  labels, tf.concat([logits_ba, logits_bn], 1), weights=weights)
nce_loss = loss_a + loss_b
print (nce_loss)

Tensor("add_4:0", shape=(), dtype=float32)


### Wrap it up with a function

In [3]:
def get_nce_negfix_loss(ha,hb,neg,temperature=1.0,weights=1.0):
    ha_nzd = tf.math.l2_normalize(ha,axis=-1)
    hb_nzd = tf.math.l2_normalize(hb,axis=-1)
    neg_nzd = tf.math.l2_normalize(neg,axis=-1)
    logits_ab = tf.matmul(ha_nzd/temperature,hb_nzd,transpose_b=True,name='logits_ab')
    logits_ba = tf.matmul(hb_nzd/temperature,ha_nzd,transpose_b=True,name='logits_ba')
    logits_an = tf.matmul(ha_nzd/temperature,neg_nzd,transpose_b=True,name='logits_an') 
    logits_bn = tf.matmul(hb_nzd/temperature,neg_nzd,transpose_b=True,name='logits_bn') 
    n_batch = tf.shape(ha)[0]
    labels = tf.one_hot(tf.range(n_batch),n_batch+n_neg,name='labels')
    loss_a = tf.losses.softmax_cross_entropy(
      labels, tf.concat([logits_ab, logits_an], 1), weights=weights)
    loss_b = tf.losses.softmax_cross_entropy(
      labels, tf.concat([logits_ba, logits_bn], 1), weights=weights)
    nce_loss = loss_a + loss_b
    return nce_loss
print ("Done.")

Done.


### Usage 1 (random feature maps)

In [4]:
tf.reset_default_graph()
sess = gpu_sess()
n_batch,dim,n_neg = 5,2,10
ha = tf.cast(tf.Variable(np.random.rand(n_batch,dim)),tf.float32,name='h1')
hb = tf.cast(tf.Variable(np.random.rand(n_batch,dim)),tf.float32,name='h2')
neg = tf.cast(tf.Variable(np.random.randn(n_neg,dim)),tf.float32,name='neg')
nce_loss = get_nce_negfix_loss(ha,hb,neg)
sess.run(tf.global_variables_initializer())
print_tf_tensor(sess,ha)
print_tf_tensor(sess,hb)
print ("Loss is [%.4f]."%(sess.run(nce_loss)))

name:[h1:0] shape:(5, 2)
[[0.156 0.677]
 [0.059 0.864]
 [0.216 0.446]
 [0.809 0.325]
 [0.084 0.933]]
name:[h2:0] shape:(5, 2)
[[0.024 0.525]
 [0.523 0.541]
 [0.93  0.811]
 [0.175 0.575]
 [0.817 0.023]]
Loss is [4.7367].


### Usage 2 (similar feature maps)

In [20]:
tf.reset_default_graph()
sess = gpu_sess()
n_batch,dim,n_neg = 5,2,5
bias = np.random.randn(n_batch,dim)
ha = tf.cast(tf.Variable(bias+0.1*np.random.randn(n_batch,dim)),tf.float32,name='h1')
hb = tf.cast(tf.Variable(bias+0.1*np.random.randn(n_batch,dim)),tf.float32,name='h2')
neg = tf.cast(tf.Variable(np.random.randn(n_neg,dim)),tf.float32,name='neg')
nce_loss = get_nce_negfix_loss(ha,hb,neg)
sess.run(tf.global_variables_initializer())
print_tf_tensor(sess,ha)
print_tf_tensor(sess,hb) 
print ("Loss is [%.4f]."%(sess.run(nce_loss)))

name:[h1:0] shape:(5, 2)
[[ 0.695 -0.506]
 [ 0.471 -1.665]
 [ 0.116  0.682]
 [ 0.663 -1.569]
 [-0.777 -1.331]]
name:[h2:0] shape:(5, 2)
[[ 0.647 -0.36 ]
 [ 0.343 -1.619]
 [ 0.048  0.702]
 [ 0.669 -1.523]
 [-0.927 -1.551]]
Loss is [3.1887].
